Uses embeddings created in previous step(s) (default : ProtTransBertBFDEmbedder , mean over first ~700 positions)

Default data: Uniref100 transmmebrane human proteins, non fragment. ~30K

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# drive.mount("/content/drive/MyDrive/Research/proteins")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install -U tqdm bio_embeddings[all]

* Use the unirpot fasta file I downloaded and uploaded to my drive

`/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta`

* Download fasta from: `https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:0.9`

In [3]:
# cd  "/content/drive/MyDrive/protein/"
# cd "/content/drive/MyDrive/Research/biodata/proteins"

# DATA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/membrane_100_whole_protbert_embed.snappy.parquet"
DATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/swp_human_viri_all_embed.parquet"

## metadata for all reviewed/swissprot human + virus proteins
METADATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/SWP_human_viruses_all.xlsx"

TARGET_COL = "human" ## use for filtering data into 1 class

In [4]:
# !wget http://data.bioembeddings.com/public/embeddings/notebooks/custom_data/tiny_sampled.fasta --output-document tiny_sampled.fasta ## tiny sample data

## https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+NOT+fragment%3Ayes+taxonomy%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22+goa%3A%28%22integral+component+of+membrane+%5B16021%5D%22%29+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:1.0
## uniref 100 transmembrane fasta, uplaoded to github: 
# !wget https://github.com/ddofer/proteinA/raw/main/UR100_Transmembrane%2BNOT_fragment%2BGO_integral.fasta --output-document Transmembrane_human_whole_100.fasta

# !wget https://github.com/ddofer/proteinA/raw/main/UR90_Transmembrane%2BNOT_fragment%2BGO_integral.fasta --output-document Transmembrane_human_whole_90.fasta

In [5]:
# MY_FASTA_PATH = "/content/drive/MyDrive/protein/tiny_sampled.fasta"
# MY_FASTA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta" # 14k sequences, with fragments, 90% id redundnancy

# MY_FASTA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_whole_100.fasta" # 30K sequences, without fragments
# MY_FASTA_PATH = "Transmembrane_human_whole_100.fasta"
# MY_FASTA_PATH = "Transmembrane_human_whole_90.fasta"

# Embed sequences in a FASTA file

In [6]:
# from bio_embeddings.embed import ProtTransBertBFDEmbedder
# from Bio import SeqIO
import pandas as pd
import numpy as np
import random

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing  import MinMaxScaler

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier   
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
# from tqdm import tqdm
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [7]:
# sequences = []
# # for record in SeqIO.parse("tiny_sampled.fasta", "fasta"): # ORIG from demo - tiny file
# for record in SeqIO.parse(MY_FASTA_PATH, "fasta"): # run on all our data
#     sequences.append(record)

In [8]:
# embedder = ProtTransBertBFDEmbedder(half_model=True)

In [9]:
df = pd.read_parquet(DATA_PATH) # numpy to pandas
 ## lengths of all the seqs
df

,ID,name,descriptions,fragment,Unknown,word_count,length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,sp|Q00266|METK1_HUMAN,sp|Q00266|METK1_HUMAN,sp|Q00266|METK1_HUMAN S-adenosylmethionine syn...,0,0,11,395,0.024226,-0.008672,0.012488,0.033686,0.005497,0.007677,-0.007326,0.018683,-0.015198,0.010641,0.036528,-0.011477,0.003085,0.089959,-0.028869,0.044023,0.065084,0.009235,-0.016290,0.297489,0.011222,0.013133,0.024094,-0.035825,-0.055611,0.027512,-0.021528,0.034961,0.021769,-0.005758,-0.009297,0.012121,0.002457,...,0.111622,-0.024995,-0.065863,-0.036349,-0.046632,0.028645,-0.054170,-0.007885,0.010368,-0.034733,-0.017257,-0.010077,0.020696,-0.013099,0.021540,0.021132,-0.049319,-0.019224,0.020732,0.002589,0.002898,0.021193,-0.013494,0.007588,0.002275,-0.071964,0.012299,0.026008,0.050261,-0.009129,0.033565,-0.109159,0.025111,-0.002302,-0.059883,0.002806,-0.013021,-0.028087,-0.024842,-0.035256
1,sp|Q8NB16|MLKL_HUMAN,sp|Q8NB16|MLKL_HUMAN,sp|Q8NB16|MLKL_HUMAN Mixed lineage kinase doma...,0,0,12,471,0.029939,0.037353,0.028395,-0.002973,0.031449,0.015336,-0.002289,-0.011943,-0.011579,0.028106,0.000403,0.006221,0.004515,0.048687,-0.012123,0.018383,0.004246,-0.005491,-0.018854,-0.458300,0.023252,-0.012191,-0.019654,-0.004178,0.003675,0.022918,0.020451,0.021829,-0.000117,-0.006812,0.025465,0.012719,-0.043243,...,-0.011569,-0.035641,0.003105,0.030347,-0.013950,0.017058,-0.035603,0.019331,0.004386,-0.005433,-0.022603,0.026585,-0.021715,-0.018533,0.003553,-0.019725,-0.034139,0.039716,-0.013684,-0.002762,0.036477,0.004674,0.001594,-0.003512,0.008066,-0.017756,0.005843,0.011560,0.021830,0.040850,0.005287,-0.033498,-0.019381,0.014456,0.030678,-0.030672,-0.000446,0.006075,-0.017941,0.017225
2,sp|O94851|MICA2_HUMAN,sp|O94851|MICA2_HUMAN,sp|O94851|MICA2_HUMAN [F-actin]-monooxygenase ...,0,0,9,1124,0.037409,0.017911,-0.007480,0.031479,0.019610,0.018506,-0.013085,0.019378,0.010587,0.003631,0.032079,-0.001212,0.005363,0.031364,-0.009566,-0.023468,0.039337,-0.000709,-0.008782,-0.153061,0.001586,0.016098,-0.015705,-0.012782,-0.071953,0.027302,0.009692,0.051514,-0.011957,-0.023841,0.027862,0.014803,-0.023104,...,0.039551,-0.029491,-0.037804,0.064259,-0.018841,-0.001436,-0.022036,0.046249,0.011425,-0.006358,-0.004876,0.020325,0.003123,-0.013926,-0.033035,-0.016771,-0.025062,0.039185,-0.005461,-0.006889,0.022403,0.020986,-0.029962,0.020149,0.029155,0.001218,-0.000874,-0.003627,0.016854,-0.030306,0.028947,-0.061823,-0.021940,-0.022406,-0.062367,-0.021113,-0.007535,0.022558,-0.056361,-0.004621
3,sp|Q8TDZ2|MICA1_HUMAN,sp|Q8TDZ2|MICA1_HUMAN,sp|Q8TDZ2|MICA1_HUMAN [F-actin]-monooxygenase ...,0,0,9,1067,0.033341,0.007557,-0.008526,0.046358,0.017429,0.025593,-0.010747,0.021048,-0.005860,0.007839,0.026372,0.001072,-0.014355,0.033230,-0.019070,-0.029877,0.038256,0.013089,-0.010168,-0.123633,0.005443,0.014957,-0.002248,-0.026796,-0.049927,0.022585,0.000278,0.066010,0.016284,-0.014193,0.039631,0.010703,-0.035178,...,0.034076,-0.024523,-0.014507,0.031881,-0.018192,0.023863,-0.003821,0.013120,0.019969,0.019480,-0.000611,0.007130,-0.007439,-0.013002,-0.044342,-0.023191,-0.031747,0.036982,-0.006467,-0.014617,0.035379,0.017270,-0.008364,0.002650,0.037445,0.001783,0.010798,0.021513,0.034119,0.013426,-0.002539,-0.058762,-0.014563,-0.004543,-0.038873,-0.039585,0.018228,0.027679,-0.027483,-0.012285
4,sp|Q9NPJ6|MED4_HUMAN,sp|Q9NPJ6|MED4_HUMAN,sp|Q9NPJ6|MED4_HUMAN Mediator of RNA polymeras...,0,0,15,270,0.008496,0.007594,0.014752,0.034230,-0.028025,0.058111,-0.017533,0.006631,0.009309,0.025146,0.071456,-0.002448,-0.004952,0.073123,-0.004334,-0.015350,0.030481,0.018443,-0.012876,-0.107023,0.045176,0.001486,-0.025902,-0.014564,-0.076639,0.021670,0.016962,0.068308,-0.002242,-0.064095,-0.000239,0.033726,0.015124,...,0.032105,-0.032264,0.004412,0.007531,

In [10]:
## metadata about all sequences, can be used to identify and to define targets/labels
df_meta = pd.read_excel(METADATA_PATH).dropna(how="all",axis=1)
df_meta

,Entry,Gene names (primary ),Entry name,Protein names,Gene names,Organism,Length,Sequence,Mass,RNA editing,Polymorphism,Non-standard residue,DNA binding,Cofactor,Binding site,Calcium binding,Activity regulation,Active site,pH dependence,Pathway,Kinetics,Keywords,Gene ontology (GO),Protein families,Pharmaceutical use,Allergenic properties,Biotechnological use,Involvement in disease,Virus hosts,Features
0,Q00266,MAT1A,METK1_HUMAN,S-adenosylmethionine synthase isoform type-1 (...,MAT1A AMS1 MATA1,Homo sapiens (Human),395,MNGPVDGLCDHSLSEGVFMFTSESVGEGHPDKICDQISDAVLDAHL...,"43,648",NaN,NaN,NaN,NaN,COFACTOR: Name=Mg(2+); Xref=ChEBI:CHEBI:18420;...,"BINDING 29; /note=""ATP""; /evidence=""ECO:0000...",NaN,NaN,NaN,NaN,PATHWAY: Amino-acid biosynthesis; S-adenosyl-L...,NaN,3D-structure;ATP-binding;Disease variant;Disul...,cytosol [GO:0005829]; ATP binding [GO:0005524]...,AdoMet synthase family,NaN,NaN,NaN,DISEASE: Methionine adenosyltransferase defici...,NaN,Beta strand (15); Binding site (9); Chain (1);...
1,Q8NB16,MLKL,MLKL_HUMAN,Mixed lineage kinase domain-like protein (hMLKL),MLKL,Homo sapiens (Human),471,MENLKHIITLGQVIHKRCEEMKYCKKQCRRLGHRVLGLIKPLEMLQ...,"54,479",NaN,NaN,NaN,NaN,NaN,"BINDING 230; /note=""ATP""; /evidence=""ECO:000...",NaN,ACTIVITY REGULATION: Activated via binding to ...,NaN,NaN,NaN,NaN,3D-structure;ATP-binding;Alternative splicing;...,cell junction [GO:0030054]; cytoplasm [GO:0005...,Protein kinase superfamily,NaN,NaN,NaN,NaN,NaN,Alternative sequence (2); Beta strand (13); Bi...
2,O94851,MICAL2,MICA2_HUMAN,[F-actin]-monooxygenase MICAL2 (EC 1.14.13.225...,MICAL2 KIAA0750 MICAL2PV1 MICAL2PV2,Homo sapiens (Human),1124,MGENEDEKQAQAGQVFENFVQASTCKGTLQAFNILTRHLDLDPLDH...,"126,689",NaN,NaN,NaN,NaN,COFACTOR: Name=FAD; Xref=ChEBI:CHEBI:57692; Ev...,"BINDING 97; /note=""FAD""; /evidence=""ECO:0000...",NaN,ACTIVITY REGULATION: Specifically inhibited by...,NaN,NaN,NaN,NaN,3D-structure;Actin-binding;Alternative splicin...,nucleus [GO:0005634]; actin binding [GO:000377...,Mical family,NaN,NaN,NaN,NaN,NaN,Alternative sequence (7); Beta strand (2); Bin...
3,Q8TDZ2,MICAL1,MICA1_HUMAN,[F-actin]-monooxygenase MICAL1 (EC 1.14.13.225...,MICAL1 MICAL NICAL,Homo sapiens (Human),1067,MASPTSTNPAHAHFESFLQAQLCQDVLSSFQELCGALGLEPGGGLP...,"117,875",NaN,NaN,NaN,NaN,COFACTOR: Name=FAD; Xref=ChEBI:CHEBI:57692; Ev...,"BINDING 95; /note=""FAD""; /evidence=""ECO:0000...",NaN,NaN,NaN,NaN,NaN,BIOPHYSICOCHEMICAL PROPERTIES: Kinetic parame...,3D-structure;Actin-binding;Alternative splicin...,actin cytoskeleton [GO:0015629]; cytoplasm [GO...,Mical family,NaN,NaN,NaN,NaN,NaN,Alternative sequence (4); Beta strand (3); Bin...
4,Q9NPJ6,MED4,MED4_HUMAN,Mediator of RNA polymerase II transcription su...,MED4 ARC36 DRIP36 VDRIP HSPC126,Homo sapiens (Human),270,MAASSSGEKEKERLGGGLGVAGGNSTRERLLSALEDLEVLSRELIE...,"29,745",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acetylation;Activator;Alternative splicing;Coi...,core mediator complex [GO:0070847]; mediator c...,Mediator complex subunit 4 family,NaN,NaN,NaN,NaN,NaN,Alternative sequence (1); Chain (1); Coiled co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37395,Q9P2T1,GMPR2,GMPR2_HUMAN,GMP reductase 2 (GMPR 2) (EC 1.7.1.7) (Guanosi...,GMPR2,Homo sapiens (Human),348,MPHIDNDVKLDFKDVLLRPKRSTLKSRSEVDLTRSFSFRNSKQTYS...,"37,874",NaN,NaN,NaN,NaN,NaN,"BINDING 78; /note=""NADP""; /evidence=""ECO:000...",NaN,NaN,"ACT_SITE 186; /note=""Thioimidate intermediate...",NaN,NaN,NaN,3D-structure;Acetylation;Alternative splicing;...,cytosol [GO:0005829]; GMP reductase complex [G...,"IMPDH/GMPR family, GuaC type 1 subfamily",NaN,NaN,NaN,NaN,NaN,Active site (2); Alternative sequence (2); Bet...
37396,Q8IUX8,EGFL6,EGFL6_HUMAN,Epidermal growth factor-like protein 6 (EGF-li...,EGFL6 MAEG PP648 UNQ281/PRO320,Homo sapiens (Human),553,MPLPWSLALPLLLSWVAGGFGNAASARHHGLLASARQPGVCHYGTK...,"61,317",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alternative splicing

In [11]:
df_meta.nunique()

Entry                     37400
Gene names  (primary )    21897
Entry name                37400
Protein names             26949
Gene names                26216
Organism                   2660
Length                     2486
Sequence                  36422
Mass                      30570
RNA editing                 151
Polymorphism                808
Non-standard residue         25
DNA binding                 649
Cofactor                   1152
Binding site               2263
Calcium binding             232
Activity regulation        1402
Active site                3085
pH dependence               285
Pathway                     745
Kinetics                    929
Keywords                  19068
Gene ontology (GO)        19132
Protein families           6365
Pharmaceutical use           48
Allergenic properties         4
Biotechnological use         23
Involvement in disease     4559
Virus hosts                 639
Features                  22965
dtype: int64

In [12]:
df_meta["Organism"].value_counts().sort_values()

Measles virus (strain Leningrad-16) (MeV) (Subacute sclerose panencephalitis virus)        1
West Nile virus (strain NY-99) (WNV) (West Nile virus (strain NY-1999))                    1
Coxsackievirus A9 (strain Griggs)                                                          1
Murine sarcoma virus NS.C58                                                                1
Tobacco mosaic virus (strain ER) (TMV)                                                     1
                                                                                       ...  
Vaccinia virus (strain Western Reserve) (VACV) (Vaccinia virus (strain WR))              230
Vaccinia virus (strain Copenhagen) (VACV)                                                257
Enterobacteria phage T4 (Bacteriophage T4)                                               268
Acanthamoeba polyphaga mimivirus (APMV)                                                  909
Homo sapiens (Human)                                                  

In [13]:
# df_meta[["Organism","Virus hosts"]].value_counts()
df_meta[["Virus hosts"]].value_counts().head(21)

Virus hosts                                                                                                                                                                                                                                                   
Homo sapiens (Human) [TaxID: 9606]                                                                                                                                                                                                                                4176
Acanthamoeba polyphaga (Amoeba) [TaxID: 5757]                                                                                                                                                                                                                      930
Escherichia coli [TaxID: 562]                                                                                                                                                                                              

In [14]:
## most frequent virus hosts, that are "close" to human - this is about half of the viruses in this dataset! 
# df_meta["virus_host_animal"] = df_meta["Virus hosts"].str.contains("Homo|Bos|Pig|Chicken|Mus|Cat|Felis|Vertebrata|Aves",na=False).astype(int)
df_meta["virus_host_animal"] = df_meta["Virus hosts"].str.contains("Homo|Human|Pig|Mouse|Vertebrata|Mammal",na=False).astype(int) ## (Musa is a type of banana tree)
df_meta.dropna(subset=["Virus hosts"])["virus_host_animal"].describe()

count    17001.000000
mean         0.454973
std          0.497983
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: virus_host_animal, dtype: float64

In [15]:
df_meta["human"] = (df_meta["Organism"] == "Homo sapiens (Human)").astype(int)
df["human"] = df_meta["human"] 
print(df["human"].mean())

0.5450802139037433


### OPT: Filter for animal host viruses only

In [16]:
print(df_meta.shape[0]," all rows")
print(df_meta.loc[(df_meta["human"] ==1) | (df_meta["virus_host_animal"]==1)].shape[0], "humans and animal host viruses only")

mask = ((df_meta["human"] ==1) | (df_meta["virus_host_animal"]==1))

## drop rows from BOTH datasets (important! and keep order!)
df_meta = df_meta[mask]
print(df_meta.shape[0])
df = df[mask]

37400  all rows
28121 humans and animal host viruses only
28121


### OPT: naive filter redundnat sequences
Drop duplicates entirely / or by subset +- reduced AA alp[habet. Not as accurate as doing BLAST, but good enough for now

In [17]:
df_meta["seq2"] = df_meta["Sequence"].str[1:350].str.replace("R","K").str.replace("I","V")
df_meta[["Sequence","seq2"] ]

print(df_meta.shape[0]) # 27949 
print(df_meta.drop_duplicates(["Sequence"]).shape[0]) # 27020
print(df_meta.drop_duplicates(["seq2"]).shape[0]) # 27020

mask_dupseq = ~df_meta["seq2"].duplicated()

## drop rows from BOTH datasets (important! and keep order!)
df_meta = df_meta[mask_dupseq]
print(df_meta.shape[0])
df = df[mask_dupseq]

df_meta.drop(columns=["seq2"],inplace=True,errors="ignore")
assert df_meta.shape[0] == df.shape[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


28121
27514
27147
27147


In [18]:
df_meta.groupby(TARGET_COL).describe().round(0)
### note length differences

Length                                   ... virus_host_animal                         
         count   mean    std  min    25%    50%  ...               std  min  25%  50%  75%  max
human                                            ...                                           
0       6866.0  498.0  586.0  6.0  173.0  330.0  ...               0.0  1.0  1.0  1.0  1.0  1.0
1      20281.0  558.0  598.0  2.0  250.0  415.0  ...               0.0  0.0  0.0  0.0  0.0  0.0

[2 rows x 16 columns]

In [72]:
y = df.set_index(['ID'])[TARGET_COL]
print(y.mean())

0.7470807087339301


## run anomaly detection models

* https://scikit-learn.org/stable/auto_examples/neighbors/plot_lof_novelty_detection.html#sphx-glr-auto-examples-neighbors-plot-lof-novelty-detection-py

* https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_anomaly_comparison.html#sphx-glr-auto-examples-miscellaneous-plot-anomaly-comparison-py

* Could also do clustering

* We will evaluate outcomes using the 

In [73]:
df.columns

Index(['ID', 'name', 'descriptions', 'fragment', 'Unknown', 'word_count',
       'length', '0', '1', '2',
       ...
       '1017', '1018', '1019', '1020', '1021', '1022', '1023', 'human',
       'anomaly_score_nn', 'anomaly_score_iso'],
      dtype='object', length=1034)

In [74]:
## .set_index(['ID'])
X = df.set_index(['ID']).drop(columns=[TARGET_COL,'descriptions', 'fragment', 'Unknown','word_count',
                                    #   'length', ## OPT : Keep protein length from raw data!
                                      ],errors="ignore").select_dtypes("number").copy()
                                      ## drop non embedding columns +- length of the raw sequence (note that initially we truncated the sequence prior tto embeddings)
## scale length feature, if using it, to not overflow

scaler = MinMaxScaler((-1,1))
X["length"] = scaler.fit_transform(X["length"].values.reshape(-1, 1))
print(X.shape)

(27147, 1027)


In [75]:
## human, virus only subsets - for use with 1 class anomlay evaluation
X_human = X.loc[y==1].copy()
print(X_human.shape[0],"human")
X_human.head()

X_virus = X[y==0].copy()
print(X_virus.shape[0],"non human")
X_virus.head()

20281 human
6866 non human


,length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,anomaly_score_nn,anomaly_score_iso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sp|P07331|MOS_MSVMH,-0.978339,0.048855,0.021544,0.020740,-0.014402,0.046853,0.040508,0.019695,-0.025294,-0.009486,-0.000762,0.046204,0.005160,-0.022279,0.061491,-0.013267,0.017396,0.049535,-0.008861,-0.030965,0.010932,0.030812,0.018935,-0.033651,-0.049144,-0.054092,0.022994,0.007121,0.024386,-0.013803,-0.063083,0.040514,0.043070,-0.023669,0.028421,0.005579,0.000066,-0.235064,-0.009086,0.041952,...,-0.007523,0.109202,-0.053900,0.018797,-0.028533,0.035553,0.002422,-0.004864,0.001271,0.003310,-0.036005,0.007539,-0.065107,-0.004181,-0.058941,0.035649,-0.010289,0.016585,0.059481,0.035868,0.035816,0.014595,0.039001,-0.028425,-0.003864,-0.015267,0.029204,0.005937,-0.007926,-0.053201,0.022910,0.019314,-0.011079,-0.028275,-0.031907,0.010559,-0.037599,-0.010028,1,0.104437
sp|P10260|PR73_MMTVB,-0.981484,-0.004199,-0.004941,0.000082,0.002436,0.033480,0.019293,-0.008199,-0.007416,-0.001613,0.023409,0.048583,-0.038850,-0.003062,0.052575,-0.002281,0.027153,0.020152,-0.017145,0.018960,-0.406880,0.009921,0.007949,0.002855,-0.007677,-0.020491,-0.001413,0.008438,0.000496,0.007109,-0.040640,0.018387,-0.004139,-0.027332,-0.018755,0.045206,0.004244,-0.031904,-0.030478,-0.007048,...,-0.004509,0.026678,-0.001001,0.004583,-0.007569,0.075567,0.008213,-0.014423,-0.012131,0.016277,0.006927,-0.002088,0.005266,-0.026586,-0.001891,0.009685,-0.017068,-0.006862,0.014003,0.014647,-0.009822,0.006346,-0.004571,-0.029451,-0.000732,0.011381,0.026594,-0.003537,0.012928,-0.038753,-0.023616,-0.000989,0.079415,-0.015444,-0.013919,0.009309,-0.042712,-0.007264,1,0.137364
sp|P04585|POL_HV1H2,-0.916560,0.050810,0.013161,-0.016064,0.022832,0.034272,0.013502,-0.051297,-0.004728,-0.017950,0.055314,0.008166,0.003342,-0.003802,0.050647,-0.010204,0.054919,0.017824,0.001328,-0.011107,-0.448903,0.041965,0.001617,0.007345,-0.009908,-0.004717,0.015841,-0.006320,-0.006123,-0.002538,-0.032017,0.020114,-0.012654,-0.034659,0.011759,0.004874,0.039439,-0.017644,-0.014065,-0.014711,...,0.010717,0.046881,0.002901,0.028611,-0.056062,0.074320,-0.010970,0.028285,-0.057961,0.033607,0.021010,-0.001269,0.033610,-0.014921,0.004357,0.008187,-0.016381,-0.040826,0.013938,0.017484,-0.008706,0.024451,0.007856,-0.030543,0.004276,0.013399,-0.018330,0.045068,0.022309,-0.012568,-0.007404,-0.012596,0.010898,-0.001397,0.009883,0.030265,-0.007418,-0.003745,1,0.120340
sp|P14078|POL_HTL1C,-0.914988,0.014617,0.001711,-0.052693,0.054337,0.025171,0.012344,-0.013477,0.009828,0.021187,0.082309,0.036408,-0.027994,-0.007911,0.035712,0.008048,-0.038144,0.040117,-0.019562,-0.048251,-0.433484,0.028325,0.015148,0.025211,-0.017798,-0.081882,0.007784,-0.045506,0.017053,0.023614,-0.057140,0.024427,0.000969,-0.002034,0.004011,0.027112,0.042974,-0.046240,-0.025596,-0.001790,...,-0.006157,0.037150,-0.008193,0.015338,-0.050481,0.137418,-0.007845,0.024093,-0.032680,0.015590,-0.010453,-0.004037,-0.043394,-0.062296,0.011454,0.010375,-0.034498,-0.030806,0.035172,0.011137,0.018285,0.006465,-0.021861,-0.015382,0.016590,0.026024,0.012516,0.023576,0.019016,-0.042919,-0.003263,-0.010319,-0.022925,-0.016796,0.007601,-0.003805,0.001366,0.004612,1,0.085697
sp|Q99IE6|POLS_RUBVO,-0.938221,0.033474,-0.009243,-0.012538,0.047398,0.038208,0.028449,-0.016432,0.022670,-0.037255,0.005775,0.044775,-0.032577,-0.043687,0.037922,-0.039216,-0.013338,0.010957,-0.001895,-0.021612,-0.065351,0.003585,-0.019016,0.032660,-0.048395,-0.019575,0.020614,-0.013248,0.011356,0.014506,-0.030519,0.042126,-0.041039,-0.024783,-0.002457,0.057137,0.025228,-0.371750,-0.009399,0.010505,...,-0.042672,-0.050227,0.022399,0.045305,0.028375,0.112338,-0.014362,0.007490,-0.0

### Supervised model

In [22]:
%%time
## in predicting human vs non human (all): 
# clf = LogisticRegression(max_iter = 400) ## 91 F1 on minority class, with human hosts
# # clf = HistGradientBoostingClassifier() ## 96 F1 & accuracy
# cv_preds_1 = cross_val_predict(estimator=clf, X=X, y=y,n_jobs=-2,cv=4)
# print(classification_report(y_true=y,y_pred=cv_preds_1))

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 21.5 µs


In [23]:
%%time
clf2 = HistGradientBoostingClassifier(max_iter=180,min_samples_leaf=15) ## 96 F1 for all & accuracy. ~ 5 min runtime. ## 94 F1 on minority class, with human hosts
cv_preds_2 = cross_val_predict(estimator=clf2, X=X, y=y,cv=5) # ,n_jobs=2
print(classification_report(y_true=y,y_pred=cv_preds_2))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      6866
           1       0.98      0.98      0.98     20281

    accuracy                           0.97     27147
   macro avg       0.96      0.96      0.96     27147
weighted avg       0.97      0.97      0.97     27147

CPU times: user 13min 22s, sys: 46.7 s, total: 14min 9s
Wall time: 7min 25s


In [129]:
%%time
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# clf3 = KNeighborsClassifier(n_neighbors =25,n_jobs=-2) ## slow
# clf3 = SVC(kernel="rbf",cache_size=1500,degree=2)
clf3 = make_pipeline(StandardScaler(), SVC(kernel="rbf",cache_size=1500)) ## 94 F1 minority class, 97 Overall F1
cv_preds_3 = cross_val_predict(estimator=clf3, X=X, y=y,n_jobs=-1,cv=5)
print(classification_report(y_true=y,y_pred=cv_preds_3))

              precision    recall  f1-score   support

           0       0.95      0.92      0.94      6866
           1       0.97      0.98      0.98     20281

    accuracy                           0.97     27147
   macro avg       0.96      0.95      0.96     27147
weighted avg       0.97      0.97      0.97     27147

CPU times: user 3.35 s, sys: 737 ms, total: 4.09 s
Wall time: 7min 35s


In [25]:
# print(classification_report(y_true=y,y_pred=cv_preds_1))

In [26]:
### mark samples that were "consistently" missclassified. We'd expect them to maybe be also anomalous
### can use just a single model, not multiple/ensemble
## ideally: y == preds1 and is equal to cv_preds_1 ; not sure on how to write  boolean condition, keep simple for now
# correct_preds = (y ==cv_preds_1) 

correct_preds = (y ==cv_preds_2) 
print(sum(correct_preds),"sum correct_preds") # 35250 for 1 model
print(len(y)-sum(correct_preds),"sum incorrect_preds") ## For human hosts, model 2 (GB),1123 incorrects (out of 29K) .  For all viri, 2150 with 1 LR model 

correct_preds
# df_meta[]

26328 sum correct_preds
819 sum incorrect_preds


0        True
1        True
2        True
3        True
4        True
         ... 
37395    True
37396    True
37397    True
37398    True
37399    True
Name: human, Length: 27147, dtype: bool

In [27]:
# incorrect_preds = ((y !=cv_preds_1) &  (y !=cv_preds_2))
# print(sum(incorrect_preds),"sum incorrect_preds") # 1357 for both models

incorrect_preds = ((y !=cv_preds_2)) 
print(sum(incorrect_preds),"sum incorrect_preds") # 1663 for GBM models

# incorrect_preds = ((y !=cv_preds_1) |  (y !=cv_preds_2))
# print(sum(incorrect_preds),"sum incorrect_preds") # 2456 for either models

incorrect_preds

819 sum incorrect_preds


0        False
1        False
2        False
3        False
4        False
         ...  
37395    False
37396    False
37397    False
37398    False
37399    False
Name: human, Length: 27147, dtype: bool

* we see there's a significant difference in lengths, accounting for both taxon and misprediction.  (but filtering for human host viruses, reduced the average length disparity)

In [29]:
df_meta["correct_preds"] = correct_preds
# df_meta.drop_duplicates(subset=["correct_preds",TARGET_COL])
# df_meta[["correct_preds",TARGET_COL,'Protein names', "Length","Organism","Virus hosts","Keywords"]]

df_meta.groupby(["correct_preds",TARGET_COL]).describe().round(0)
## 

Length                            ... virus_host_animal                    
                       count   mean    std  min    25%  ...               min  25%  50%  75%  max
correct_preds human                                     ...                                      
False         0        459.0  288.0  429.0  9.0  104.0  ...               1.0  1.0  1.0  1.0  1.0
              1        360.0  275.0  330.0  8.0  115.0  ...               0.0  0.0  0.0  0.0  0.0
True          0       6407.0  513.0  593.0  6.0  179.0  ...               1.0  1.0  1.0  1.0  1.0
              1      19921.0  563.0  600.0  2.0  256.0  ...               0.0  0.0  0.0  0.0  0.0

[4 rows x 16 columns]

In [30]:
df_errors = df_meta.loc[df_meta["correct_preds"]==False].copy()#[["correct_preds",TARGET_COL,'Protein names', "Length","Organism","Virus hosts","Keywords"]]
print(df_errors.shape[0],"rows with mispredictions",df_errors[TARGET_COL].value_counts(normalize=True) )
df_errors[["correct_preds",TARGET_COL,'Protein names', "Length","Organism","Virus hosts","Keywords"]]

819 rows with mispredictions 0    0.56044
1    0.43956
Name: human, dtype: float64


,correct_preds,human,Protein names,Length,Organism,Virus hosts,Keywords
12,False,1,Protein-arginine deiminase type-3 (EC 3.5.3.15...,664,Homo sapiens (Human),NaN,3D-structure;Calcium;Cytoplasm;Disease variant...
26,False,1,Meiosis-specific with OB domain-containing pro...,442,Homo sapiens (Human),NaN,Alternative splicing;Chromosome;Cytoplasm;DNA-...
162,False,1,Protein MFI (Mitochondrial fission factor inte...,313,Homo sapiens (Human),NaN,Alternative splicing;Cytoplasm;Membrane;Mitoch...
221,False,0,Shutoff protein (100 kDa protein) (p100K) (100...,805,Human adenovirus C serotype 2 (HAdV-2) (Human ...,Homo sapiens (Human) [TaxID: 9606],Chaperone;Eukaryotic host gene expression shut...
222,False,0,Shutoff protein (100 kDa protein) (p100K) (100...,782,Human adenovirus A serotype 12 (HAdV-12) (Huma...,Homo sapiens (Human) [TaxID: 9606],Chaperone;Eukaryotic host gene expression shut...
...,...,...,...,...,...,...,...
37088,False,0,Protein Vpu (U ORF protein) (Viral protein U),85,Human immunodeficiency virus type 1 group O (i...,Homo sapiens (Human) [TaxID: 9606],AIDS;Apoptosis;Host membrane;Host-virus intera...
37124,False,0,Uncharacterized 10.4 kDa protein,90,Vaccinia virus (strain Copenhagen) (VACV),Homo sapiens (Human) [TaxID: 9606],NaN
37345,False,1,Guided entry of tail-anchored proteins factor ...,174,Homo sapiens (Human),NaN,3D-structure;Alternative splicing;Coiled coil;...
37363,False,0,Envelope glycoprotein N,91,Human herpesvirus 1 (strain 17) (HHV-1) (Human...,Homo sapiens (Human) [TaxID: 9606],Disulfide bond;Host Golgi apparatus;Host membr...


In [31]:
print(df_meta.groupby(["correct_preds"])["Virus hosts"].value_counts(normalize=True).sort_values(ascending=False).round(2).head(4))

correct_preds  Virus hosts                                                                                                                                                                                                                             
False          Homo sapiens (Human) [TaxID: 9606]                                                                                                                                                                                                          0.64
True           Homo sapiens (Human) [TaxID: 9606]                                                                                                                                                                                                          0.54
               Ornithodoros (relapsing fever ticks) [TaxID: 6937]; Phacochoerus aethiopicus (Warthog) [TaxID: 85517]; Phacochoerus africanus (Warthog) [TaxID: 41426]; Potamochoerus larvatus (Bushpig) [TaxID: 273792]; Sus scrofa (Pig) [TaxID

In [33]:
df_meta["Virus hosts"].nunique()

116

In [35]:
df_errors.loc[df_errors[TARGET_COL]==0][["correct_preds",TARGET_COL,'Protein names', "Length","Organism","Virus hosts","Keywords"]]

,correct_preds,human,Protein names,Length,Organism,Virus hosts,Keywords
221,False,0,Shutoff protein (100 kDa protein) (p100K) (100...,805,Human adenovirus C serotype 2 (HAdV-2) (Human ...,Homo sapiens (Human) [TaxID: 9606],Chaperone;Eukaryotic host gene expression shut...
222,False,0,Shutoff protein (100 kDa protein) (p100K) (100...,782,Human adenovirus A serotype 12 (HAdV-12) (Huma...,Homo sapiens (Human) [TaxID: 9606],Chaperone;Eukaryotic host gene expression shut...
573,False,0,Phosphoprotein (Protein P) (Protein M1),297,Rabies virus (strain Nishigahara RCEH) (RABV),Homo sapiens (Human) [TaxID: 9606]; Mammalia [...,Alternative initiation;Chaperone;Cytoplasmic i...
574,False,0,Phosphoprotein (Protein P) (Protein M1),297,Rabies virus (strain China/DRV) (RABV),Homo sapiens (Human) [TaxID: 9606]; Mammalia [...,Alternative initiation;Chaperone;Cytoplasmic i...
903,False,0,Replicase polyprotein 1ab (ORF1ab polyprotein)...,3961,Porcine reproductive and respiratory syndrome ...,Sus scrofa (Pig) [TaxID: 9823],ATP-binding;Endonuclease;Helicase;Host cytopla...
...,...,...,...,...,...,...,...
36983,False,0,Virion-associated protein (Vap) (Protein 3) (P3),129,Cauliflower mosaic virus (strain Strasbourg) (...,Arabidopsis thaliana (Mouse-ear cress) [TaxID:...,3D-structure;Coiled coil;Disulfide bond;Host c...
37088,False,0,Protein Vpu (U ORF protein) (Viral protein U),85,Human immunodeficiency virus type 1 group O (i...,Homo sapiens (Human) [TaxID: 9606],AIDS;Apoptosis;Host membrane;Host-virus intera...
37124,False,0,Uncharacterized 10.4 kDa protein,90,Vaccinia virus (strain Copenhagen) (VACV),Homo sapiens (Human) [TaxID: 9606],NaN
37363,False,0,Envelope glycoprotein N,91,Human herpesvirus 1 (strain 17) (HHV-1) (Human...,Homo sapiens (Human) [TaxID: 9606],Disulfide bond;Host Golgi apparatus;Host membr...


In [36]:
df_errors.loc[df_errors[TARGET_COL]==0]["Organism"].value_counts().head(12)

Human herpesvirus 8 type P (isolate GK18) (HHV-8) (Kaposi's sarcoma-associated herpesvirus)    23
Human cytomegalovirus (strain AD169) (HHV-5) (Human herpesvirus 5)                             19
Human cytomegalovirus (strain Merlin) (HHV-5) (Human herpesvirus 5)                            16
Vaccinia virus (strain Copenhagen) (VACV)                                                      14
Epstein-Barr virus (strain B95-8) (HHV-4) (Human herpesvirus 4)                                13
Human herpesvirus 6A (strain Uganda-1102) (HHV-6 variant A) (Human B lymphotropic virus)       11
Epstein-Barr virus (strain GD1) (HHV-4) (Human herpesvirus 4)                                  11
Human adenovirus C serotype 2 (HAdV-2) (Human adenovirus 2)                                    11
Epstein-Barr virus (strain AG876) (HHV-4) (Human herpesvirus 4)                                10
Human herpesvirus 6B (strain Z29) (HHV-6 variant B) (Human B lymphotropic virus)               10
Fowlpox virus (strai

In [37]:
df_errors.loc[df_errors[TARGET_COL]==1]# human proteins mispredicted as being viruses - TODO: look at go keywords (cytoplasm? ) 

,Entry,Gene names (primary ),Entry name,Protein names,Gene names,Organism,Length,Sequence,Mass,RNA editing,Polymorphism,Non-standard residue,DNA binding,Cofactor,Binding site,Calcium binding,Activity regulation,Active site,pH dependence,Pathway,Kinetics,Keywords,Gene ontology (GO),Protein families,Pharmaceutical use,Allergenic properties,Biotechnological use,Involvement in disease,Virus hosts,Features,virus_host_animal,human,correct_preds
12,Q9ULW8,PADI3,PADI3_HUMAN,Protein-arginine deiminase type-3 (EC 3.5.3.15...,PADI3 PAD3 PDI3,Homo sapiens (Human),664,MSLQRIVRVSLEHPTSAVCVAGVETLVDIYGSVPEGTEMFEVYGTP...,"74,743",NaN,NaN,NaN,NaN,COFACTOR: Name=Ca(2+); Xref=ChEBI:CHEBI:29108;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3D-structure;Calcium;Cytoplasm;Disease variant...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,Protein arginine deiminase family,NaN,NaN,NaN,DISEASE: Uncombable hair syndrome 1 (UHS1) [MI...,NaN,Beta strand (42); Chain (1); Helix (13); Natur...,0,1,False
26,Q8N635,MEIOB,MEIOB_HUMAN,Meiosis-specific with OB domain-containing pro...,MEIOB C16orf73,Homo sapiens (Human),442,MANSFAARIFTTLSDLQTNMANLKVIGIVIGKTDVKGFPDRKNIGS...,"49,313",NaN,NaN,NaN,"DNA_BIND 167..272; /note=""OB""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alternative splicing;Chromosome;Cytoplasm;DNA-...,chromosome [GO:0005694]; cytoplasm [GO:0005737...,MEIOB family,NaN,NaN,NaN,DISEASE: Spermatogenic failure 22 (SPGF22) [MI...,NaN,Alternative sequence (1); Chain (1); DNA bindi...,0,1,False
162,Q8NCR3,MFI,MFI_HUMAN,Protein MFI (Mitochondrial fission factor inte...,MFI C11orf65,Homo sapiens (Human),313,MPWKEESEFTKQDKAARVIQQAWKSFLNVAIFQHFKSLIDLRRQGE...,"36,720",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alternative splicing;Cytoplasm;Membrane;Mitoch...,cytosol [GO:0005829]; mitochondrial outer memb...,NaN,NaN,NaN,NaN,NaN,NaN,Alternative sequence (2); Chain (1),0,1,False
241,Q13296,SCGB2A2,SG2A2_HUMAN,Mammaglobin-A (Mammaglobin-1) (Secretoglobin f...,SCGB2A2 MGB1 UGB2,Homo sapiens (Human),93,MKLLMVLMLAALSQHCYAGSGCPLLENVISKTINPQVSKTEYKELL...,"10,499",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alternative splicing;Glycoprotein;Reference pr...,extracellular space [GO:0005615]; androgen rec...,"Secretoglobin family, Lipophilin subfamily",NaN,NaN,NaN,NaN,NaN,Alternative sequence (1); Chain (1); Glycosyla...,0,1,False
279,Q9Y5M6,OCLM,TISR_HUMAN,Oculomedin (Trabecular meshwork-inducible stre...,OCLM,Homo sapiens (Human),44,MGMYPPLLLKIYLSRHISILFYLKILYKSGIIWLSWYSFILLVL,"5,321",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reference proteome,visual perception [GO:0007601],NaN,NaN,NaN,NaN,NaN,NaN,Chain (1),0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36754,Q8TBK6,ZCCHC10,ZCH10_HUMAN,Zinc finger CCHC domain-containing protein 10,ZCCHC10,Homo sapiens (Human),192,MATPMHRLIARRQAFDTELQPVKTFWILIQPSIVISEANKQHVRCQ...,"20,967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alternative splicing;Metal-binding;Reference p...,nucleic acid binding [GO:0003676]; zinc ion bi...,NaN,NaN,NaN,NaN,NaN,NaN,Alternative sequence (1); Chain (1); Compositi...,0,1,False
36804,A0AVI4,TMEM129,TM129_HUMAN,E3 ubiquitin-protein ligase TM129 (EC 2.3.2.27...,TMEM129,Homo sapiens (Human),362,MDSPEVTFTLAYLVFAVCFVFTPNEFHAAGLTVQNLLSGWLGSEDA...,"40,464",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PATHWAY: Protein modification; protein ubiquit...,NaN,Alternative splicing;Endoplasmic reticulum;Mem...,endoplasmic reticulum [GO:0005783]; endoplasmi...,TMEM129 family,NaN,NaN,NaN,NaN,NaN,Alternative sequence (2); Chain (1); Natural v...,0,1,False
36806,H0YL14,TMEM250,TM250_HUMAN,Transmembrane protein 250 (Herpes virus UL25-b...,TMEM250 C9orf69,Homo sapiens (Human),139,MPVMPIPRRVRSFHGPHTTCLHAACGPVRASHLARTKYNNFDVYIK...,"16,083",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cell cycle;Cytoplasm;Host-virus interaction;Me...,cell division site [GO:0032153]; cytoplasm [GO...,NaN,NaN,NaN,NaN,NaN,NaN,Chain (1); Transmembrane (2

In [38]:
df_errors.columns

Index(['Entry', 'Gene names  (primary )', 'Entry name', 'Protein names',
       'Gene names', 'Organism', 'Length', 'Sequence', 'Mass', 'RNA editing',
       'Polymorphism', 'Non-standard residue', 'DNA binding', 'Cofactor',
       'Binding site', 'Calcium binding', 'Activity regulation', 'Active site',
       'pH dependence', 'Pathway', 'Kinetics', 'Keywords',
       'Gene ontology (GO)', 'Protein families', 'Pharmaceutical use',
       'Allergenic properties', 'Biotechnological use',
       'Involvement in disease', 'Virus hosts', 'Features',
       'virus_host_animal', 'human', 'correct_preds'],
      dtype='object')

In [39]:
df_errors.loc[df_errors[TARGET_COL]==1]['Gene names  (primary )'].dropna().values

array(['PADI3', 'MEIOB', 'MFI', 'SCGB2A2', 'OCLM', 'RTL4', 'POLR2H',
       'RPRM', 'SMIM30', 'ST20-AS1', 'TNP2', 'PGBD3', 'TPRKB', 'PRPF8',
       'RNF125', 'POLR1E', 'TBC1D28', 'TERT', 'TMEM167A', 'SMIM33',
       'SLFN5', 'SMIM3', 'MYMX', 'SMIM1', 'SUFU', 'TMEM185B', 'PEX11G',
       'SLFN12', 'MRPL53', 'RTRAF', 'SLC66A1L', 'POLR3GL', 'TYW3',
       'SIVA1', 'POP5', 'RNASE9', 'SPANXA2-OT1', 'PSMG4', 'PBK', 'SDHAF1',
       'TMEM102', 'P3R3URF', 'TMEM185A', 'SRP14', 'RNF4', 'SMCO3',
       'SPATA45', 'SMIM35', 'UGGT2', 'RAG2', 'RNASE13', 'RNASE12',
       'MRPS30', 'SGCB', 'PSMF1', 'SNAPC3', 'XCL2', 'SMIM10L2A', 'RTL6',
       'RNASE2', 'PAIP2', 'PFN3', 'RNF227', 'RNF6', 'PJVK', 'CT83',
       'P2RX4', 'PEA15', 'PKHD1L1', 'LYRM1', 'MNDA', 'TMEM167B', 'IFI27',
       'OOSP4B', 'OLFML2B', 'OSCP1', 'SPATA18', 'HRAS', 'LIF', 'MSH4',
       'RO60', 'MTRNR2L3', 'TEPP', 'TSHB', 'SPP2', 'SPTSSB', 'TSLP',
       'TMEM71', 'TMEM117', 'TMEM240', 'UPK3A', 'FYTTD1', 'WFDC9',
       'NXPE1', 'UTF1

In [40]:
# df_meta.drop_duplicates(subset=["Sequence"]).shape[0] ## 36422 if drop 100 % duplicate sequences, out of 37400 total
# df_meta

### output predictions/errors

In [41]:
# df_meta.to_csv("protein_meta_human.csv.gz",index=False,compression="gzip") ## output with errors. Hopefully lists will be parsed ok... # first, with all

df_meta.to_csv("protein_errorAnalysis_human.csv.gz",index=False,compression="gzip") ## output with errors. Hopefully lists will be parsed ok...

## Anomaly models

In [42]:
%%time
### slow - ~ 15 min
clf = LocalOutlierFactor(n_neighbors=25,n_jobs=-1)  ## default: minkowski, euclidean 
preds = clf.fit_predict(X)
print(preds.mean(),"mean preds") # 0.93

df["anomaly_score_nn"] = preds


0.9377463439790769 mean preds
CPU times: user 15min 59s, sys: 869 ms, total: 16min
Wall time: 15min 56s


In [43]:
%%time
# # clf = LocalOutlierFactor(n_neighbors=30,n_jobs=-2 ,metric="l1")
# # preds = clf.fit_predict(X)
# # print(preds.mean(),"mean preds")
# # df["anomaly_score_nn_L1"] = preds

# clf = LocalOutlierFactor(n_neighbors=25,n_jobs=-2 ,metric="cosine")
# preds = clf.fit_predict(X)
# print(preds.mean(),"mean preds")
# df["anomaly_score_nn_cos"] = preds

# preds

KeyboardInterrupt: ignored

In [44]:
# clf.negative_outlier_factor_

# negative_outlier_factor_ndarray of shape (n_samples,)
# The opposite LOF of the training samples. The higher, the more normal. Inliers tend to have a LOF score close to 1 (negative_outlier_factor_ close to -1), while outliers tend to have a larger LOF score.

# The local outlier factor (LOF) of a sample captures its supposed ‘degree of abnormality’. It is the average of the ratio of the local reachability density of a sample and those of its k-nearest neighbors.


AttributeError: ignored

In [ ]:
preds.mean()

In [45]:
%%time
clf_iso = IsolationForest(max_samples=1100,n_jobs=-1,bootstrap=True)
clf_iso.fit(X)

df["anomaly_score_iso"] = clf_iso.decision_function(X) # get averaged, probalistic" score

CPU times: user 31.9 s, sys: 1.13 s, total: 33 s
Wall time: 24.2 s


In [46]:
##  The anomaly score of the input samples. The lower, the more abnormal. Negative scores represent outliers, positive scores represent inliers.

df[["anomaly_score_iso","anomaly_score_nn"]].describe().round(3)

,anomaly_score_iso,anomaly_score_nn
count,27147.000,27147.000
mean,0.101,0.938
std,0.040,0.347
min,-0.158,-1.000
25%,0.086,1.000
50%,0.111,1.000
75%,0.128,1.000
max,0.163,1.000


In [76]:
anomaly_score_cols = ["anomaly_score_iso","anomaly_score_nn"
                                # ,"anomaly_score_nn_cos"
                                ] 
                        # "anomaly_score_nn_L1",

In [77]:
df2 = df[["ID","descriptions","length","Unknown","word_count"] +anomaly_score_cols].copy()
        #   "anomaly_score_iso","anomaly_score_nn", # "anomaly_score_nn_cos",#"anomaly_score_nn_L1",
          
df2.sort_values(anomaly_score_cols,inplace=True)
## FLIP direction so that higher is more anomalous, at least for iso_forest, for easier interpretation/correlation
for c in anomaly_score_cols:
    df2[c] = -1*df2[c]    
# df2["anomaly_score_iso"] = -1*df2["anomaly_score_iso"]
# df2["anomaly_score_nn"] = -1*df2["anomaly_score_nn"]
df2


,ID,descriptions,length,Unknown,word_count,anomaly_score_iso,anomaly_score_nn
10299,sp|Q6ZSN1|YI023_HUMAN,sp|Q6ZSN1|YI023_HUMAN Putative uncharacterized...,163,1,10,0.158297,-1
20269,sp|Q156A1|ATX8_HUMAN,sp|Q156A1|ATX8_HUMAN Ataxin-8 OS=Homo sapiens ...,80,0,8,0.146472,1
34009,sp|P23210|VP19_HHV1K,sp|P23210|VP19_HHV1K Triplex capsid protein VP...,6,0,15,0.145160,1
35532,sp|P0DPR3|TRDD1_HUMAN,sp|P0DPR3|TRDD1_HUMAN T cell receptor delta di...,2,0,13,0.144764,1
1651,sp|P0CAN2|VFE66_ASFWA,sp|P0CAN2|VFE66_ASFWA Uncharacterized protein ...,26,1,14,0.143354,1
...,...,...,...,...,...,...,...
34906,sp|P29475|NOS1_HUMAN,"sp|P29475|NOS1_HUMAN Nitric oxide synthase, br...",1434,0,11,-0.159169,-1
15423,sp|Q14527|HLTF_HUMAN,sp|Q14527|HLTF_HUMAN Helicase-like transcripti...,1009,0,10,-0.159199,-1
37167,sp|Q5FWF4|ZRAB3_HUMAN,sp|Q5FWF4|ZRAB3_HUMAN DNA annealing helicase a...,1079,0,13,-0.159340,-1
18597,sp|Q16832|DDR2_HUMAN,sp|Q16832|DDR2_HUMAN Discoidin domain-containi...,855,0,11,-0.159529,-1


In [78]:
df2.describe().round(2)

,length,Unknown,word_count,anomaly_score_iso,anomaly_score_nn
count,27147.00,27147.00,27147.00,27147.00,27147.00
mean,542.92,0.04,11.55,-0.10,-0.94
std,595.23,0.19,2.11,0.04,0.35
min,2.00,0.00,7.00,-0.16,-1.00
25%,225.00,0.00,10.00,-0.13,-1.00
50%,395.00,0.00,11.00,-0.11,-1.00
75%,649.00,0.00,13.00,-0.09,-1.00
max,34350.00,1.00,20.00,0.16,1.00


#### Top rows = predict to be most anomalous (by isolation forest model)

In [79]:
df2.loc[df2["anomaly_score_nn"]==1].shape[0]

845

In [80]:
## check correlation of the 2 models - how many mutual/shared anomalous predictions?
df2.loc[(df2["anomaly_score_nn"]==1) & (df2["anomaly_score_iso"]>=0)].shape[0]

170

In [81]:
### anomalous according to all models
df2["anomalous"] = (df2[anomaly_score_cols]>0).all(axis=1) #df2.loc[(df2[anomaly_score_cols]>0).all(axis=1)]
print("Most anomalous rows:")
print(df2[df2["anomalous"]].describe().round(2))
df2[df2["anomalous"]]

Most anomalous rows:
        length  Unknown  word_count  anomaly_score_iso  anomaly_score_nn
count   170.00   170.00      170.00             170.00             170.0
mean    290.81     0.12       12.12               0.05               1.0
std     376.96     0.32        2.61               0.04               0.0
min       2.00     0.00        7.00               0.00               1.0
25%      73.00     0.00       10.00               0.01               1.0
50%     225.00     0.00       12.00               0.04               1.0
75%     317.75     0.00       13.00               0.06               1.0
max    2850.00     1.00       19.00               0.15               1.0


,ID,descriptions,length,Unknown,word_count,anomaly_score_iso,anomaly_score_nn,anomalous
20269,sp|Q156A1|ATX8_HUMAN,sp|Q156A1|ATX8_HUMAN Ataxin-8 OS=Homo sapiens ...,80,0,8,0.146472,1,True
34009,sp|P23210|VP19_HHV1K,sp|P23210|VP19_HHV1K Triplex capsid protein VP...,6,0,15,0.145160,1,True
35532,sp|P0DPR3|TRDD1_HUMAN,sp|P0DPR3|TRDD1_HUMAN T cell receptor delta di...,2,0,13,0.144764,1,True
1651,sp|P0CAN2|VFE66_ASFWA,sp|P0CAN2|VFE66_ASFWA Uncharacterized protein ...,26,1,14,0.143354,1,True
35565,sp|Q69566|U88_HHV6U,sp|Q69566|U88_HHV6U Uncharacterized protein U8...,413,1,13,0.140440,1,True
...,...,...,...,...,...,...,...,...
12470,sp|O76100|OR7AA_HUMAN,sp|O76100|OR7AA_HUMAN Olfactory receptor 7A10 ...,309,0,10,0.001290,1,True
14696,sp|P09430|STP1_HUMAN,sp|P09430|STP1_HUMAN Spermatid nuclear transit...,55,0,12,0.001051,1,True
752,sp|P28085|SH_MUMP4,sp|P28085|SH_MUMP4 Small hydrophobic protein O...,57,0,13,0.000942,1,True
15114,sp|P01889|HLAB_HUMAN,sp|P01889|HLAB_HUMAN HLA class I histocompatib...,362,0,15,0.000353,1,True


In [82]:
df2[df2["anomalous"]].describe().round(2)

,length,Unknown,word_count,anomaly_score_iso,anomaly_score_nn
count,170.00,170.00,170.00,170.00,170.0
mean,290.81,0.12,12.12,0.05,1.0
std,376.96,0.32,2.61,0.04,0.0
min,2.00,0.00,7.00,0.00,1.0
25%,73.00,0.00,10.00,0.01,1.0
50%,225.00,0.00,12.00,0.04,1.0
75%,317.75,0.00,13.00,0.06,1.0
max,2850.00,1.00,19.00,0.15,1.0


In [83]:
print("non anomalous rows/majority:")
df2[~df2["anomalous"]].describe().round(2)

non anomalous rows/majority:


,length,Unknown,word_count,anomaly_score_iso,anomaly_score_nn
count,26977.00,26977.00,26977.00,26977.00,26977.00
mean,544.51,0.04,11.55,-0.10,-0.95
std,596.02,0.19,2.10,0.04,0.31
min,10.00,0.00,7.00,-0.16,-1.00
25%,227.00,0.00,10.00,-0.13,-1.00
50%,397.00,0.00,11.00,-0.11,-1.00
75%,651.00,0.00,13.00,-0.09,-1.00
max,34350.00,1.00,20.00,0.16,1.00


In [84]:
print("Anomalous IDs:")

df2[df2["anomalous"]]["ID"].values

Anomalous IDs:


array(['sp|Q156A1|ATX8_HUMAN', 'sp|P23210|VP19_HHV1K',
       'sp|P0DPR3|TRDD1_HUMAN', 'sp|P0CAN2|VFE66_ASFWA',
       'sp|Q69566|U88_HHV6U', 'sp|P0CG48|UBC_HUMAN',
       'sp|P03915|NU5M_HUMAN', 'sp|P62945|RL41_HUMAN',
       'sp|Q6ZRX8|YL004_HUMAN', 'sp|P0DOY5|HD101_HUMAN',
       'sp|P04553|HSP1_HUMAN', 'sp|P0C215|P12I_HTL1A',
       'sp|H0YM25|GG6LV_HUMAN', 'sp|P0CAN1|VFE66_ASFP4',
       'sp|P0DPI4|TDB01_HUMAN', 'sp|E2RYF6|MUC22_HUMAN',
       'sp|P51861|CDR1_HUMAN', 'sp|Q17RH7|TPRXL_HUMAN',
       'sp|P26553|VE5_HPV51', 'sp|P03905|NU4M_HUMAN',
       'sp|P03886|NU1M_HUMAN', 'sp|Q9QJ13|B8_HHV6Z',
       'sp|P03901|NU4LM_HUMAN', 'sp|P0CG47|UBB_HUMAN',
       'sp|Q9NRI7|PPY2_HUMAN', 'sp|P01858|TUFT_HUMAN',
       'sp|P20511|YVAB_VACCC', 'sp|P00846|ATP6_HUMAN',
       'sp|Q8N1N5|CRPAK_HUMAN', 'sp|Q07283|TRHY_HUMAN',
       'sp|O00370|LORF2_HUMAN', 'sp|P02728|GLEM_HUMAN',
       'sp|Q9NZW4|DSPP_HUMAN', 'sp|P03923|NU6M_HUMAN',
       'sp|X6R8D5|GUCNB_HUMAN', 'sp|P25187|VP7_ROTH4',
    

In [85]:
str(df2[df2["anomalous"]]["ID"].values).strip().replace("'","").replace("\n","")

'[sp|Q156A1|ATX8_HUMAN sp|P23210|VP19_HHV1K sp|P0DPR3|TRDD1_HUMAN sp|P0CAN2|VFE66_ASFWA sp|Q69566|U88_HHV6U sp|P0CG48|UBC_HUMAN sp|P03915|NU5M_HUMAN sp|P62945|RL41_HUMAN sp|Q6ZRX8|YL004_HUMAN sp|P0DOY5|HD101_HUMAN sp|P04553|HSP1_HUMAN sp|P0C215|P12I_HTL1A sp|H0YM25|GG6LV_HUMAN sp|P0CAN1|VFE66_ASFP4 sp|P0DPI4|TDB01_HUMAN sp|E2RYF6|MUC22_HUMAN sp|P51861|CDR1_HUMAN sp|Q17RH7|TPRXL_HUMAN sp|P26553|VE5_HPV51 sp|P03905|NU4M_HUMAN sp|P03886|NU1M_HUMAN sp|Q9QJ13|B8_HHV6Z sp|P03901|NU4LM_HUMAN sp|P0CG47|UBB_HUMAN sp|Q9NRI7|PPY2_HUMAN sp|P01858|TUFT_HUMAN sp|P20511|YVAB_VACCC sp|P00846|ATP6_HUMAN sp|Q8N1N5|CRPAK_HUMAN sp|Q07283|TRHY_HUMAN sp|O00370|LORF2_HUMAN sp|P02728|GLEM_HUMAN sp|Q9NZW4|DSPP_HUMAN sp|P03923|NU6M_HUMAN sp|X6R8D5|GUCNB_HUMAN sp|P25187|VP7_ROTH4 sp|Q9BWJ2|RHAS1_HUMAN sp|Q04840|VP7_ROTHX sp|P50774|VE5_HPV70 sp|Q96MT4|CF195_HUMAN sp|Q6DPU3|M2_I02A1 sp|P00414|COX3_HUMAN sp|P0DPF3|NBPF9_HUMAN sp|Q6DPQ7|M2_I02A7 sp|Q6ZRP5|YD019_HUMAN sp|P04554|PRM2_HUMAN sp|P20523|YVAN_VACCC sp|P064

In [86]:
list(df2.descriptions.head(7))

['sp|Q6ZSN1|YI023_HUMAN Putative uncharacterized protein FLJ45355 OS=Homo sapiens OX=9606 PE=2 SV=1',
 'sp|Q156A1|ATX8_HUMAN Ataxin-8 OS=Homo sapiens OX=9606 GN=ATXN8 PE=1 SV=1',
 'sp|P23210|VP19_HHV1K Triplex capsid protein VP19C (Fragment) OS=Human herpesvirus 1 (strain KOS) OX=10306 GN=UL38 PE=3 SV=1',
 'sp|P0DPR3|TRDD1_HUMAN T cell receptor delta diversity 1 OS=Homo sapiens OX=9606 GN=TRDD1 PE=1 SV=1',
 'sp|P0CAN2|VFE66_ASFWA Uncharacterized protein E66L OS=African swine fever virus (isolate Warthog/Namibia/Wart80/1980) OX=561444 GN=War-146 PE=3 SV=1',
 'sp|P0C728|LF3_EBVG Uncharacterized protein LF3 OS=Epstein-Barr virus (strain GD1) OX=10376 GN=LF3 PE=3 SV=1',
 'sp|Q69566|U88_HHV6U Uncharacterized protein U88 OS=Human herpesvirus 6A (strain Uganda-1102) OX=10370 GN=U88 PE=4 SV=1']

In [87]:
[i +"_HUMAN" for i in df2[df2["anomalous"]]["ID"].values]

['sp|Q156A1|ATX8_HUMAN_HUMAN',
 'sp|P23210|VP19_HHV1K_HUMAN',
 'sp|P0DPR3|TRDD1_HUMAN_HUMAN',
 'sp|P0CAN2|VFE66_ASFWA_HUMAN',
 'sp|Q69566|U88_HHV6U_HUMAN',
 'sp|P0CG48|UBC_HUMAN_HUMAN',
 'sp|P03915|NU5M_HUMAN_HUMAN',
 'sp|P62945|RL41_HUMAN_HUMAN',
 'sp|Q6ZRX8|YL004_HUMAN_HUMAN',
 'sp|P0DOY5|HD101_HUMAN_HUMAN',
 'sp|P04553|HSP1_HUMAN_HUMAN',
 'sp|P0C215|P12I_HTL1A_HUMAN',
 'sp|H0YM25|GG6LV_HUMAN_HUMAN',
 'sp|P0CAN1|VFE66_ASFP4_HUMAN',
 'sp|P0DPI4|TDB01_HUMAN_HUMAN',
 'sp|E2RYF6|MUC22_HUMAN_HUMAN',
 'sp|P51861|CDR1_HUMAN_HUMAN',
 'sp|Q17RH7|TPRXL_HUMAN_HUMAN',
 'sp|P26553|VE5_HPV51_HUMAN',
 'sp|P03905|NU4M_HUMAN_HUMAN',
 'sp|P03886|NU1M_HUMAN_HUMAN',
 'sp|Q9QJ13|B8_HHV6Z_HUMAN',
 'sp|P03901|NU4LM_HUMAN_HUMAN',
 'sp|P0CG47|UBB_HUMAN_HUMAN',
 'sp|Q9NRI7|PPY2_HUMAN_HUMAN',
 'sp|P01858|TUFT_HUMAN_HUMAN',
 'sp|P20511|YVAB_VACCC_HUMAN',
 'sp|P00846|ATP6_HUMAN_HUMAN',
 'sp|Q8N1N5|CRPAK_HUMAN_HUMAN',
 'sp|Q07283|TRHY_HUMAN_HUMAN',
 'sp|O00370|LORF2_HUMAN_HUMAN',
 'sp|P02728|GLEM_HUMAN_HUMAN',
 '

In [88]:
## check whether fragment is common
df2.loc[df2["descriptions"].str.contains("Fragment")].shape[0]

242

In [89]:
df2[anomaly_score_cols].corr().round(2)

,anomaly_score_iso,anomaly_score_nn
anomaly_score_iso,1.00,0.17
anomaly_score_nn,0.17,1.00


In [90]:
df2.corr().round(3)

,length,Unknown,word_count,anomaly_score_iso,anomaly_score_nn,anomalous
length,1.000,-0.106,-0.054,-0.192,-0.033,-0.034
Unknown,-0.106,1.000,0.064,0.169,0.043,0.032
word_count,-0.054,0.064,1.000,0.111,0.083,0.021
anomaly_score_iso,-0.192,0.169,0.111,1.000,0.168,0.291
anomaly_score_nn,-0.033,0.043,0.083,0.168,1.000,0.443
anomalous,-0.034,0.032,0.021,0.291,0.443,1.000


In [91]:
print(df2.loc[(df2["anomaly_score_nn"]==1) & (df2["anomaly_score_iso"]>=0.05)].shape[0])
df2.loc[(df2["anomaly_score_nn"]==1) & (df2["anomaly_score_iso"]>=0.05)]["ID"].values

68


array(['sp|Q156A1|ATX8_HUMAN', 'sp|P23210|VP19_HHV1K',
       'sp|P0DPR3|TRDD1_HUMAN', 'sp|P0CAN2|VFE66_ASFWA',
       'sp|Q69566|U88_HHV6U', 'sp|P0CG48|UBC_HUMAN',
       'sp|P03915|NU5M_HUMAN', 'sp|P62945|RL41_HUMAN',
       'sp|Q6ZRX8|YL004_HUMAN', 'sp|P0DOY5|HD101_HUMAN',
       'sp|P04553|HSP1_HUMAN', 'sp|P0C215|P12I_HTL1A',
       'sp|H0YM25|GG6LV_HUMAN', 'sp|P0CAN1|VFE66_ASFP4',
       'sp|P0DPI4|TDB01_HUMAN', 'sp|E2RYF6|MUC22_HUMAN',
       'sp|P51861|CDR1_HUMAN', 'sp|Q17RH7|TPRXL_HUMAN',
       'sp|P26553|VE5_HPV51', 'sp|P03905|NU4M_HUMAN',
       'sp|P03886|NU1M_HUMAN', 'sp|Q9QJ13|B8_HHV6Z',
       'sp|P03901|NU4LM_HUMAN', 'sp|P0CG47|UBB_HUMAN',
       'sp|Q9NRI7|PPY2_HUMAN', 'sp|P01858|TUFT_HUMAN',
       'sp|P20511|YVAB_VACCC', 'sp|P00846|ATP6_HUMAN',
       'sp|Q8N1N5|CRPAK_HUMAN', 'sp|Q07283|TRHY_HUMAN',
       'sp|O00370|LORF2_HUMAN', 'sp|P02728|GLEM_HUMAN',
       'sp|Q9NZW4|DSPP_HUMAN', 'sp|P03923|NU6M_HUMAN',
       'sp|X6R8D5|GUCNB_HUMAN', 'sp|P25187|VP7_ROTH4',
    

In [92]:
df2.head(50)["ID"].values

array(['sp|Q6ZSN1|YI023_HUMAN', 'sp|Q156A1|ATX8_HUMAN',
       'sp|P23210|VP19_HHV1K', 'sp|P0DPR3|TRDD1_HUMAN',
       'sp|P0CAN2|VFE66_ASFWA', 'sp|P0C728|LF3_EBVG',
       'sp|Q69566|U88_HHV6U', 'sp|Q8AZK7|EBNA5_EBVB9',
       'sp|A0A1B0GTR4|SPRR5_HUMAN', 'sp|P0CG48|UBC_HUMAN',
       'sp|Q5TZK3|FAM74_HUMAN', 'sp|Q1HVI8|EBNA5_EBVA8',
       'sp|P03915|NU5M_HUMAN', 'sp|P62945|RL41_HUMAN',
       'sp|Q6ZRX8|YL004_HUMAN', 'sp|P03181|BHLF1_EBVB9',
       'sp|P0DOY5|HD101_HUMAN', 'sp|P04553|HSP1_HUMAN',
       'sp|P08519|APOA_HUMAN', 'sp|Q3LI63|KR201_HUMAN',
       'sp|Q3LI61|KR202_HUMAN', 'sp|P0DPF2|NBPFK_HUMAN',
       'sp|P0C215|P12I_HTL1A', 'sp|H0YM25|GG6LV_HUMAN',
       'sp|Q7Z4W3|KR193_HUMAN', 'sp|Q6L8H1|KRA54_HUMAN',
       'sp|Q3SYF9|KR197_HUMAN', 'sp|Q6L8G5|KR510_HUMAN',
       'sp|Q8WY50|PLAC4_HUMAN', 'sp|P15941|MUC1_HUMAN',
       'sp|P0CAN1|VFE66_ASFP4', 'sp|P04280|PRP1_HUMAN',
       'sp|Q9BYR3|KRA44_HUMAN', 'sp|Q8IUB9|KR191_HUMAN',
       'sp|P0DPI4|TDB01_HUMAN', 'sp|Q8N7P7|

In [93]:
df2.tail(100)["ID"].values

array(['sp|Q8NHP6|MSPD2_HUMAN', 'sp|P0DP91|ERPG3_HUMAN',
       'sp|Q4VNC1|AT134_HUMAN', 'sp|Q96NW4|ANR27_HUMAN',
       'sp|Q13972|RGRF1_HUMAN', 'sp|Q6VVX0|CP2R1_HUMAN',
       'sp|Q9H4A4|AMPB_HUMAN', 'sp|Q86TI2|DPP9_HUMAN',
       'sp|Q5T848|GP158_HUMAN', 'sp|Q2TBA0|KLH40_HUMAN',
       'sp|Q32P41|TRM5_HUMAN', 'sp|Q8NER1|TRPV1_HUMAN',
       'sp|Q92995|UBP13_HUMAN', 'sp|P51589|CP2J2_HUMAN',
       'sp|P14778|IL1R1_HUMAN', 'sp|P87503|DPOL_ADE04',
       'sp|Q7RTP6|MICA3_HUMAN', 'sp|D0EZM8|NS1_HBOC1',
       'sp|Q8IUH4|ZDH13_HUMAN', 'sp|Q8NBA8|DTWD2_HUMAN',
       'sp|P24462|CP3A7_HUMAN', 'sp|Q15080|NCF4_HUMAN',
       'sp|Q96SW2|CRBN_HUMAN', 'sp|P25481|VE1_HPV2A',
       'sp|P07099|HYEP_HUMAN', 'sp|Q5TGY1|TMCO4_HUMAN',
       'sp|Q8IW41|MAPK5_HUMAN', 'sp|Q5VWQ0|RSBN1_HUMAN',
       'sp|P03111|VE1_HPV1', 'sp|P34925|RYK_HUMAN',
       'sp|Q9ULI2|RIMKB_HUMAN', 'sp|P54317|LIPR2_HUMAN',
       'sp|Q81999|VE1_HPV72', 'sp|P18433|PTPRA_HUMAN',
       'sp|P38935|SMBP2_HUMAN', 'sp|Q93100|KPBB_H

## Anomaly/Novelty prediction models
* Train model on humans +- anomalies, then get predictions on other class and see how many are anomalous 


* LocalOutlierFactor with **Cosine** distance, and 1NN gets **61%** of viruses as novel
* 1 Class SVM - RBF - gets 91%  (with 11 min runtime) 
* LocalOutlierFactor(n_neighbors=25, "minkowski") gets 100-68% = 32% of viruses markes as novelty/outliers
`LocalOutlierFactor.predict: Predict the labels (1 inlier, -1 outlier) `
* Iso forest does poorly
* Anomaly approach (mixed data ) gets ~ 40%


In [114]:
%%time
# ## novelty model
## minkowski does worse and is much slower
# clf_normal = LocalOutlierFactor(n_neighbors=15,n_jobs=-1,novelty=True)  ## default: minkowski, euclidean  ## contamination =0.005
# clf_normal.fit(X_human)

# preds_anomaly = clf_normal.predict(X_virus) # (1 inlier, -1 outlier)
# print(preds_anomaly.mean())
# print(100*(preds_anomaly== -1).mean(), "% tagged as anomaly") ## 15%

0.6810369938829013
0.15948150305854938 % tagged as anomaly
CPU times: user 18min, sys: 896 ms, total: 18min 1s
Wall time: 9min 14s


In [123]:
%%time
clf_normal_cos = LocalOutlierFactor(n_neighbors=1,n_jobs=-1,novelty=True,metric = "cosine")  ## default: minkowski, euclidean  ## less nearest neighbouyrs, better perf
clf_normal_cos.fit(X_human)
preds_anomaly_cos = clf_normal_cos.predict(X_virus)
print(100*((preds_anomaly_cos== -1).mean()), "% tagged as anomaly") ##59% with 3 nearest neighbours, 61% with 1=NN

61.36032624526653 % tagged as anomaly
CPU times: user 1min 35s, sys: 1.66 s, total: 1min 36s
Wall time: 58.8 s


In [115]:
# clf_normal_cos = LocalOutlierFactor(n_neighbors=25,n_jobs=-1,novelty=True,metric = "cosine")  ## default: minkowski, euclidean  ## contamination =0.005
# clf_normal_cos.fit(X_human)
# preds_anomaly_cos = clf_normal_cos.predict(X_virus)
# print(100*(preds_anomaly_cos== -1).mean(), "% tagged as anomaly") ## 54%

54.06350131080687 % tagged as anomaly


In [116]:
# %%time
# clf_normal_cos = LocalOutlierFactor(n_neighbors=15,n_jobs=-1,novelty=True,metric = "cosine",p=1)  ## default: minkowski, euclidean  ## contamination =0.005
# clf_normal_cos.fit(X_human)
# preds_anomaly_cos = clf_normal_cos.predict(X_virus)
# print(100*(preds_anomaly_cos== -1).mean(), "% tagged as anomaly") ## %55

55.228662976988055 % tagged as anomaly


In [130]:
%%time
# clf_normal_svm = OneClassSVM(kernel="rbf",cache_size=3200)  ## 91 % , 12 min
clf_normal_svm = make_pipeline(StandardScaler(), OneClassSVM(kernel="rbf",cache_size=2500)) ## 86.7% with scaler, 12.5 min
clf_normal_svm.fit(X_human)
preds_anomaly_svm = clf_normal_svm.predict(X_virus)
print(100*((preds_anomaly_svm== -1).mean()), "% tagged as anomaly") ## linear kernel : 55.38% detected (long runtime) , RBF kernel:91.15 % , 11 min runtime ,  

86.29478590154383 % tagged as anomaly
CPU times: user 12min 30s, sys: 2.19 s, total: 12min 32s
Wall time: 12min 28s


In [121]:
%%time
### iso forest is meant for anomaly, not novelty. Very bad performance in novelty scenario naively - gets 2% (note - in that scenario it wasn't trained with viri data, i.e innappropiate OOD shift)
clf_normal_iso = IsolationForest(max_samples=1100,n_jobs=-1,bootstrap=False) #,contamination =0.005)
clf_normal_iso.fit(X_human)
preds_anomaly_iso = clf_normal_iso.predict(X_virus)
print(100*(preds_anomaly_iso== -1).mean(), "% tagged as anomaly") ## %8 with auto contamination. (Does far worse if using contamination =0.005)

9.539761141858433 % tagged as anomaly
CPU times: user 17.4 s, sys: 885 ms, total: 18.3 s
Wall time: 11.5 s


#### Try trainign mixed model on all data, with anomaly fraction set by expected %
*i.e anomaly , rather than novelty, detection

* Isolation forest marks about 40% of the viruses as outliers


https://scikit-learn.org/0.19/auto_examples/covariance/plot_outlier_detection.html#sphx-glr-auto-examples-covariance-plot-outlier-detection-py

https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_anomaly_comparison.html#


In [103]:
X_virus.shape[0]/X.shape[0]

0.25291929126606993

In [105]:
%%time
clf_mixed = IsolationForest(max_samples=1500,n_jobs=-1,bootstrap=True,contamination =(X_virus.shape[0]/X.shape[0]))
clf_mixed.fit(X)

preds_anomaly_mix = clf_mixed.predict(X_virus)
print(preds_anomaly_mix.mean())

print(100*(preds_anomaly_mix== -1).mean(), "% tagged as anomaly")

0.40489367899796097
CPU times: user 39 s, sys: 957 ms, total: 40 s
Wall time: 29.7 s
